In [1]:
import optuna
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split


In [2]:
# Cargar el conjunto de datos MNIST
mnist = fetch_openml('mnist_784', version=1)
X, y = mnist["data"], mnist["target"]

# Normalizar las imágenes de 0 a 1
X = X.astype('float32') / 255.0

# Convertir las etiquetas a enteros y luego a categóricas (one-hot encoding)
y = to_categorical(y.astype('int'))

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


h:\Anaconda\envs\IA\lib\site-packages\sklearn\datasets\_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [3]:
def create_model(trial):
    model = Sequential()
    model.add(Flatten(input_shape=(784,)))

    # Sugerir el número de capas ocultas y las unidades en cada capa
    n_layers = trial.suggest_int('n_layers', 1, 3)
    for i in range(n_layers):
        num_units = trial.suggest_int(f'n_units_l{i}', 32, 128)
        model.add(Dense(num_units, activation=trial.suggest_categorical(f'activation_l{i}', ['relu', 'tanh'])))

    model.add(Dense(10, activation='softmax'))
    
    # Compilar el modelo
    optimizer = trial.suggest_categorical('optimizer', ['adam', 'rmsprop', 'sgd'])
    model.compile(optimizer=optimizer, 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    
    return model

def objective(trial):
    # Crear el modelo
    model = create_model(trial)

    # Sugerir el tamaño del batch size y el número de epochs
    batch_size = trial.suggest_int('batch_size', 32, 128)
    epochs = trial.suggest_int('epochs', 10, 20)

    # Entrenar el modelo
    history = model.fit(X_train, y_train, 
                        validation_data=(X_test, y_test), 
                        batch_size=batch_size, 
                        epochs=epochs, 
                        verbose=0)

    # Obtener la mejor precisión de validación
    accuracy = max(history.history['val_accuracy'])
    return accuracy


In [4]:
# Crear un estudio para realizar la optimización
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

# Imprimir los mejores hiperparámetros y el mejor rendimiento
print(f"Best trial: {study.best_trial.value}")
print(f"Best parameters: {study.best_trial.params}")


[I 2024-09-06 08:44:17,130] A new study created in memory with name: no-name-2f1b8237-c6c5-4941-859d-c7089e9d67df
[I 2024-09-06 08:44:35,010] Trial 0 finished with value: 0.9720714092254639 and parameters: {'n_layers': 3, 'n_units_l0': 81, 'activation_l0': 'tanh', 'n_units_l1': 45, 'activation_l1': 'tanh', 'n_units_l2': 99, 'activation_l2': 'tanh', 'optimizer': 'adam', 'batch_size': 121, 'epochs': 12}. Best is trial 0 with value: 0.9720714092254639.
[I 2024-09-06 08:44:59,009] Trial 1 finished with value: 0.9489285945892334 and parameters: {'n_layers': 3, 'n_units_l0': 61, 'activation_l0': 'tanh', 'n_units_l1': 76, 'activation_l1': 'relu', 'n_units_l2': 128, 'activation_l2': 'relu', 'optimizer': 'sgd', 'batch_size': 96, 'epochs': 16}. Best is trial 0 with value: 0.9720714092254639.
[I 2024-09-06 08:45:14,842] Trial 2 finished with value: 0.9351428747177124 and parameters: {'n_layers': 2, 'n_units_l0': 61, 'activation_l0': 'relu', 'n_units_l1': 63, 'activation_l1': 'relu', 'optimizer': 

KeyboardInterrupt: 

In [ ]:
best_params = study.best_trial.params

# Crear el modelo final con los mejores hiperparámetros
model = Sequential()
model.add(Flatten(input_shape=(784,)))

for i in range(best_params['n_layers']):
    model.add(Dense(best_params[f'n_units_l{i}'], activation=best_params[f'activation_l{i}']))

model.add(Dense(10, activation='softmax'))

model.compile(optimizer=best_params['optimizer'], 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Entrenar el modelo final
model.fit(X_train, y_train, 
          validation_data=(X_test, y_test), 
          batch_size=best_params['batch_size'], 
          epochs=best_params['epochs'], 
          verbose=1)
